## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Talnakh,RU,69.4865,88.3972,-0.80,95,100,11.32,overcast clouds
1,1,Nanortalik,GL,60.1432,-45.2371,11.82,60,3,29.77,clear sky
2,2,Matadi,CD,-5.8167,13.4500,75.74,90,89,3.58,light rain
3,3,Lorengau,PG,-2.0226,147.2712,81.73,81,97,9.26,overcast clouds
4,4,Yellowknife,CA,62.4560,-114.3525,-8.72,86,99,2.91,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10) 

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Matadi,CD,-5.8167,13.4500,75.74,90,89,3.58,light rain
3,3,Lorengau,PG,-2.0226,147.2712,81.73,81,97,9.26,overcast clouds
6,6,Boma,GH,7.0805,-2.1697,78.73,25,11,2.21,few clouds
7,7,Avarua,CK,-21.2078,-159.7750,82.45,78,40,9.22,scattered clouds
10,10,Riberalta,BO,-10.9833,-66.1000,87.48,57,80,1.41,broken clouds
12,12,Ilhabela,BR,-23.7781,-45.3581,75.00,82,89,6.08,overcast clouds
23,23,Rikitea,PF,-23.1203,-134.9692,78.78,72,69,11.36,light rain
24,24,Georgetown,MY,5.4112,100.3354,77.90,89,20,0.00,few clouds
28,28,Kirakira,SB,-10.4544,161.9205,81.46,81,100,10.36,moderate rain
36,36,Townsville,AU,-19.2500,146.8000,76.08,83,20,0.00,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                195
City                   195
Country                195
Lat                    195
Lng                    195
Max Temp               195
Humidity               195
Cloudiness             195
Wind Speed             195
Current Description    195
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(axis="index", how="any")
print(len(clean_df))
clean_df.head()

195


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Matadi,CD,-5.8167,13.4500,75.74,90,89,3.58,light rain
3,3,Lorengau,PG,-2.0226,147.2712,81.73,81,97,9.26,overcast clouds
6,6,Boma,GH,7.0805,-2.1697,78.73,25,11,2.21,few clouds
7,7,Avarua,CK,-21.2078,-159.7750,82.45,78,40,9.22,scattered clouds
10,10,Riberalta,BO,-10.9833,-66.1000,87.48,57,80,1.41,broken clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Matadi,CD,75.74,light rain,-5.8167,13.4500,
3,Lorengau,PG,81.73,overcast clouds,-2.0226,147.2712,
6,Boma,GH,78.73,few clouds,7.0805,-2.1697,
7,Avarua,CK,82.45,scattered clouds,-21.2078,-159.7750,
10,Riberalta,BO,87.48,broken clouds,-10.9833,-66.1000,
12,Ilhabela,BR,75.00,overcast clouds,-23.7781,-45.3581,
23,Rikitea,PF,78.78,light rain,-23.1203,-134.9692,
24,Georgetown,MY,77.90,few clouds,5.4112,100.3354,
28,Kirakira,SB,81.46,moderate rain,-10.4544,161.9205,
36,Townsville,AU,76.08,few clouds,-19.2500,146.8000,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

hotel_df.head(10)

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Matadi,CD,75.74,light rain,-5.8167,13.4500,Hotel Vivi Palace
3,Lorengau,PG,81.73,overcast clouds,-2.0226,147.2712,Lorengau Harbourside Hotel
6,Boma,GH,78.73,few clouds,7.0805,-2.1697,Obaatanpa Hotel
7,Avarua,CK,82.45,scattered clouds,-21.2078,-159.7750,Paradise Inn
10,Riberalta,BO,87.48,broken clouds,-10.9833,-66.1000,Hotel Colonial
12,Ilhabela,BR,75.00,overcast clouds,-23.7781,-45.3581,Ilha Flat Hotel
23,Rikitea,PF,78.78,light rain,-23.1203,-134.9692,People ThankYou
24,Georgetown,MY,77.90,few clouds,5.4112,100.3354,Cititel Penang
28,Kirakira,SB,81.46,moderate rain,-10.4544,161.9205,SanBiz Lodge
36,Townsville,AU,76.08,few clouds,-19.2500,146.8000,Rydges Southbank Townsville


In [9]:
# 7. Drop the rows where there is no Hotel Name.
print(hotel_df.count())
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
print(len(clean_hotel_df))
clean_hotel_df.head(10)

City                   195
Country                195
Max Temp               195
Current Description    195
Lat                    195
Lng                    195
Hotel Name             195
dtype: int64
195


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Matadi,CD,75.74,light rain,-5.8167,13.4500,Hotel Vivi Palace
3,Lorengau,PG,81.73,overcast clouds,-2.0226,147.2712,Lorengau Harbourside Hotel
6,Boma,GH,78.73,few clouds,7.0805,-2.1697,Obaatanpa Hotel
7,Avarua,CK,82.45,scattered clouds,-21.2078,-159.7750,Paradise Inn
10,Riberalta,BO,87.48,broken clouds,-10.9833,-66.1000,Hotel Colonial
12,Ilhabela,BR,75.00,overcast clouds,-23.7781,-45.3581,Ilha Flat Hotel
23,Rikitea,PF,78.78,light rain,-23.1203,-134.9692,People ThankYou
24,Georgetown,MY,77.90,few clouds,5.4112,100.3354,Cititel Penang
28,Kirakira,SB,81.46,moderate rain,-10.4544,161.9205,SanBiz Lodge
36,Townsville,AU,76.08,few clouds,-19.2500,146.8000,Rydges Southbank Townsville


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))